In [1]:
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dill
from typing import List
import scipy.stats as sts
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

import util
from pathlib import Path
from class_manipulates_path import ManipulatePath
from class_preprocessing_refactor import Preprocessing
from class_format_data import FormatData
util.init()

current_path = sys.path[0]

In [2]:
def plot_variable(df, column_name, intervalos=None):
    """
    Plota um gráfico de dispersão para uma variável de um DataFrame.

    Parâmetros:
        - df: DataFrame pandas contendo os dados.
        - column_name: Nome da coluna que você deseja plotar.
        - intervalos: Lista de intervalos para plotar linhas verticais.
    """
    plt.figure(figsize=(20, 10))
    plt.scatter(df.index.values, df[column_name].values, color='darkcyan', alpha=0.5)
    plt.xlabel('Índice')
    plt.ylabel(column_name)
    plt.title(f'Gráfico de dispersão para a variável "{column_name}"')


    # Plotar linhas verticais para cada intervalo
    if intervalos:
        for intervalo in intervalos:
            plt.axvline(x=intervalo, linestyle='--', color='red')

    plt.show()

In [3]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [4]:
manipulate_path = ManipulatePath()

preprocessing = Preprocessing()

format_data = FormatData()

path_raw_data = manipulate_path.get_path_raw_data()

abnormal_classes_codes = [1, 2, 5, 6, 7, 8]

In [5]:
# Gets all real instances but maintains only those with any type of undesirable event
real_instances = pd.DataFrame(class_and_file_generator(path_raw_data, 
                                                       real=True,
                                                       simulated=True, 
                                                       drawn=True),
                              columns=['class_code', 'instance_path'])
real_instances = real_instances.loc[real_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

In [6]:
falha = 8
falha_2 = 108

In [7]:
real_instances[real_instances["class_code"] == falha]

,class_code,instance_path
853,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
854,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
855,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
856,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
857,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
...,...,...
929,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
930,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
931,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
932,8,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...


In [8]:
real_instances[real_instances["class_code"] == falha]["instance_path"].values

array([WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/8/SIMULATED_00001.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/8/SIMULATED_00002.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/8/SIMULATED_00003.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/8/SIMULATED_00004.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/8/SIMULATED_00005.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/8/SIMULATED_00006.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Fina

In [9]:
file_paths = real_instances[real_instances["class_code"] == falha]["instance_path"].values
df_output = pd.DataFrame()

for file_path in file_paths:
    df_data = pd.read_csv(file_path)

    df_data = FormatData.set_index_dataframe(df_data=df_data,
                                         column_name="timestamp")
    quantidade = ((df_data['class'] == falha) | (df_data['class'] == falha_2)).sum()
    if quantidade != 0:
        print(f"1 se repete {quantidade}")
        print(f"Index min {df_data.index.min()} index max {df_data.index.max()}")

    df_output = pd.concat([df_output, df_data], axis=0)

1 se repete 25199
Index min 2018-05-17 22:56:06 index max 2018-05-18 06:26:04
1 se repete 25199
Index min 2018-05-18 00:32:29 index max 2018-05-18 08:02:27
1 se repete 25199
Index min 2018-05-18 00:52:57 index max 2018-05-18 08:22:55
1 se repete 25199
Index min 2018-05-18 01:19:03 index max 2018-05-18 08:49:01
1 se repete 25199
Index min 2018-05-18 01:57:26 index max 2018-05-18 09:27:24
1 se repete 25200
Index min 2018-05-18 02:29:53 index max 2018-05-18 09:59:52
1 se repete 25199
Index min 2018-05-18 15:17:29 index max 2018-05-18 22:47:27
1 se repete 25199
Index min 2018-05-18 15:36:50 index max 2018-05-18 23:06:48
1 se repete 25199
Index min 2018-05-18 15:52:51 index max 2018-05-18 23:22:49
1 se repete 25199
Index min 2018-05-18 16:07:25 index max 2018-05-18 23:37:23
1 se repete 25199
Index min 2018-05-18 16:27:41 index max 2018-05-18 23:57:39
1 se repete 25199
Index min 2018-05-18 16:48:45 index max 2018-05-19 00:18:43
1 se repete 25199
Index min 2018-05-18 17:11:27 index max 2018-0

c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Periodo\2_Data_Science\2_Trabalho_Final\3_3W\util\class_format_data.py:112: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index)


1 se repete 25199
Index min 2018-05-18 20:18:55 index max 2018-05-19 03:48:53
1 se repete 25199
Index min 2018-05-18 21:18:05 index max 2018-05-19 04:48:03
1 se repete 25199
Index min 2018-05-18 21:45:04 index max 2018-05-19 05:15:02
1 se repete 25199
Index min 2018-05-19 00:44:59 index max 2018-05-19 08:14:57
1 se repete 25199
Index min 2018-05-19 01:19:06 index max 2018-05-19 08:49:04
1 se repete 25199
Index min 2018-05-19 01:53:53 index max 2018-05-19 09:23:51
1 se repete 25199
Index min 2018-05-19 02:51:48 index max 2018-05-19 10:21:46
1 se repete 25199
Index min 2018-05-19 04:40:51 index max 2018-05-19 12:10:49
1 se repete 25199
Index min 2018-05-19 04:59:10 index max 2018-05-19 12:29:08
1 se repete 25199
Index min 2018-05-19 05:29:51 index max 2018-05-19 12:59:49
1 se repete 25199
Index min 2018-05-19 06:08:10 index max 2018-05-19 13:38:08
1 se repete 25199
Index min 2018-05-19 06:40:37 index max 2018-05-19 14:10:35
1 se repete 25199
Index min 2018-05-19 07:12:31 index max 2018-0

In [10]:
df_output

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,
2018-05-17 22:56:06,27897490.0,19824230.0,125.685900,4059666.0,97.55283,NaN,NaN,NaN,0
2018-05-17 22:56:07,27897450.0,19824230.0,125.685900,4059666.0,97.55283,NaN,NaN,NaN,0
2018-05-17 22:56:08,27897360.0,19824230.0,125.685900,4059666.0,97.55283,NaN,NaN,NaN,0
2018-05-17 22:56:09,27897430.0,19824230.0,125.685900,4059666.0,97.55282,NaN,NaN,NaN,0
2018-05-17 22:56:10,27897500.0,19824230.0,125.685900,4059666.0,97.55282,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
2018-05-19 19:01:05,33586320.0,28076270.0,3.253794,4002285.0,37.74498,NaN,NaN,NaN,108
2018-05-19 19:01:06,33586440.0,28076420.0,3.253723,4002285.0,38.21421,NaN,NaN,NaN,108
2018-05-19 19:01:07,33586900.0,28076490.0,3.253835,4002278.0,38.75061,NaN,NaN,NaN,108


In [11]:
df_output = df_output[~df_output.index.duplicated()]

In [12]:
df_output.shape

(139976, 9)

In [13]:
df_aux = df_output.drop(columns=["QGL", "T-JUS-CKGL", "P-JUS-CKGL"])
df_aux

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
timestamp,,,,,,
2018-05-17 22:56:06,27897490.0,19824230.0,125.685900,4059666.0,97.55283,0
2018-05-17 22:56:07,27897450.0,19824230.0,125.685900,4059666.0,97.55283,0
2018-05-17 22:56:08,27897360.0,19824230.0,125.685900,4059666.0,97.55283,0
2018-05-17 22:56:09,27897430.0,19824230.0,125.685900,4059666.0,97.55282,0
2018-05-17 22:56:10,27897500.0,19824230.0,125.685900,4059666.0,97.55282,0
...,...,...,...,...,...,...
2018-05-19 19:01:05,33586320.0,28076270.0,3.253794,4002285.0,37.74498,108
2018-05-19 19:01:06,33586440.0,28076420.0,3.253723,4002285.0,38.21421,108
2018-05-19 19:01:07,33586900.0,28076490.0,3.253835,4002278.0,38.75061,108


In [14]:
for column_name in df_aux.columns:
    percent = Preprocessing.check_null_dataframe(df_data=df_output, column_name=column_name)
    print(f"{column_name} null data is {percent}")

P-PDG null data is 0.0
P-TPT null data is 0.0
T-TPT null data is 0.0
P-MON-CKP null data is 0.0
T-JUS-CKP null data is 0.0
class null data is 0.0


In [15]:
df_aux = df_aux.dropna()
df_aux.shape

(139976, 6)

In [16]:
(df_aux['class'] == 0).sum() / df_aux.shape[0]

0.025740126878893526

# KNN

In [17]:
contagem_classes = df_aux['class'].value_counts()

# Calcular a porcentagem de cada classe
porcentagem_classes = contagem_classes / len(df_aux) * 100
porcentagem_classes

class
8      77.596874
108    19.829114
0       2.574013
Name: count, dtype: float64

In [18]:
# Separar as features (X) e os rótulos (y)
X = df_aux.drop('class', axis=1)
y = df_aux['class']

In [19]:
X

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP
timestamp,,,,,
2018-05-17 22:56:06,27897490.0,19824230.0,125.685900,4059666.0,97.55283
2018-05-17 22:56:07,27897450.0,19824230.0,125.685900,4059666.0,97.55283
2018-05-17 22:56:08,27897360.0,19824230.0,125.685900,4059666.0,97.55283
2018-05-17 22:56:09,27897430.0,19824230.0,125.685900,4059666.0,97.55282
2018-05-17 22:56:10,27897500.0,19824230.0,125.685900,4059666.0,97.55282
...,...,...,...,...,...
2018-05-19 19:01:05,33586320.0,28076270.0,3.253794,4002285.0,37.74498
2018-05-19 19:01:06,33586440.0,28076420.0,3.253723,4002285.0,38.21421
2018-05-19 19:01:07,33586900.0,28076490.0,3.253835,4002278.0,38.75061


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [22]:
modelo = KNeighborsClassifier(n_neighbors=5)

In [23]:
modelo.fit(X_train, y_train)

KNeighborsClassifier()

In [24]:
y_pred = modelo.predict(X_test)

In [25]:
accuracy_score(y_test, y_pred)

0.9981901745529017

In [26]:
precision = precision_score(y_test,  y_pred, average=None)
recall = recall_score(y_test,  y_pred, average=None)
f1 = f1_score(y_test,  y_pred, average=None)

for i in range(len(precision)):
    print(f'Classe {porcentagem_classes.index.values[i]}:')
    print(f'Precision: {precision[i]:.3f}')
    print(f'Recall: {recall[i]:.3f}')
    print(f'F1-score: {f1[i]:.3f}\n')

Classe 8:
Precision: 0.997
Recall: 0.999
F1-score: 0.998

Classe 108:
Precision: 0.999
Recall: 0.999
F1-score: 0.999

Classe 0:
Precision: 0.996
Recall: 0.995
F1-score: 0.995



In [27]:
confusion_matrix(y_test.values, y_pred)

array([[ 1080,     0,     1],
       [    0, 32555,    30],
       [    3,    42,  8282]], dtype=int64)

# MLP

In [55]:
modelo = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [56]:
modelo.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [57]:
y_pred = modelo.predict(X_test)

In [58]:
accuracy_score(y_test, y_pred)

0.9899507060700593